# Data Loading, Storage, and File Formats

Input and output typically falls into a few main categories: reading text files and other more efficient on-disk formats, loading data from databases, and interacting with network sources like web APIs.

## Reading and Writing Data in Text Format

pandas features a number of functions for reading tabular data as a DataFrame object. __read_csv__ and __read_table__ are likely the ones you'll use the most.

Because of how messy data in the real world can be, some of the data loading functions (especially __read_csv__) habe grown very complex in their options over time. It's normal to feel overwhelmed by the number of different parameters(__read_csv__ has over 50)

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../examples/ex1.csv')

In [3]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
pd.read_table('../examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


File without a header row:

In [5]:
pd.read_csv('../examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_csv('../examples/ex2.csv', names=['a','b','c','d','message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
names = ['a','b','c','d','message']

In [8]:
pd.read_csv('../examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [9]:
parsed = pd.read_csv('../examples/csv_mindex.csv', 
                    index_col=['key1','key2'])

In [10]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

If no comma:

In [11]:
result = pd.read_table('../examples/ex3.txt', sep='\s+') # \s+ indicates whitespace

In [12]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [13]:
pd.read_csv('../examples/ex4.csv', skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [14]:
result = pd.read_csv('../examples/ex5.csv')

In [15]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [16]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [17]:
result = pd.read_csv('../examples/ex5.csv', na_values=['NULL'])

In [18]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [19]:
sentinels = {'message': ['foo','NA'], 'something': ['two']}

In [20]:
pd.read_csv('../examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in Pieces

In [21]:
pd.options.display.max_rows = 10 # we only read the first and last part of the data file

In [22]:
result = pd.read_csv('../examples/ex6.csv')

In [23]:
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [24]:
pd.read_csv('../examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [25]:
chunker = pd.read_csv('../examples/ex6.csv', chunksize=1000)

In [26]:
chunker # ...

### Writing Data to Text Format

In [27]:
data = pd.read_csv('../examples/ex5.csv')

In [28]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Using DataFrame's __to_csv__ method, we can write the data out to a comma separated file:

In [29]:
data.to_csv('../examples/out.csv')

Other delimiters can be used, of course (writing to __sys.stdout__ so it prints the text result to the console):

In [30]:
import sys

In [31]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Missing values appear as empty strings in the output. You might want to denote them by some other sentinel value:

In [32]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [33]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [34]:
data.to_csv(sys.stdout, index=False, columns=['a','b','c']) # just a subset

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series also has a __to_csv__ method:

In [35]:
dates = pd.date_range('1/1/2000', periods=7)

In [36]:
import numpy as np

In [37]:
ts = pd.Series(np.arange(7), index=dates)

In [38]:
ts.to_csv('../examples/tseries.csv')

### Working with Delimited Formats

It is possible to load most forms of tabular data from disk using functions like __pandas.read_table__. In some cases, however, some manual processing may be necessary.  

In [39]:
import csv

In [40]:
f = open('../examples/ex7.csv')

In [41]:
reader = csv.reader(f)

In [42]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [43]:
with open('../examples/ex7.csv') as f:
    lines = list(csv.reader(f))

In [44]:
header, values = lines[0], lines[1:]

In [45]:
data_dict = {h: v for h, v in zip(header,zip(*values))}

In [46]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

### JSON Data

JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTML request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV. 
There are several Python libraries for reading and writing JSON data. We'll use __json__ here, as it is built into the Python standard library. To convert a JSON string to Python form, use __json.loads__:

In [47]:
obj = """
{"name": "Wes",
"places_lived": ["United States", "Spain", "Germany"],
"pets": null,
"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus","Zuko"]},
            {"name": "Katie", "age": 38, "pets": ["Sixes","Stache","Cisco"]}]
}
"""

In [48]:
import json

In [49]:
result = json.loads(obj)

In [50]:
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pets': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

__json.dumps__, on the other hand, converts a Python object back to JSON:

In [51]:
asjson = json.dumps(result)

Transformation of the loaded JSON file into a DataFrame

In [52]:
siblings = pd.DataFrame(result['siblings'], columns=['name','age'])

In [53]:
siblings

,name,age
0,Scott,30
1,Katie,38


The __pandas.read_json__ can automatically convert JSON datasets in specific arrangements into a Series or DataFrame:

In [54]:
data = pd.read_json('../examples/example.json')

In [55]:
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [56]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [57]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### XML and HTML: Web Scraping

Python has many libraries for reading and writing in the ubiquitous HTML and XML formats. Examples include lxml (http://lxml.de), Beautiful Soup, and html5lib. 

pandas has a built-in function, __read_html__, which uses libraries like lxml and Beautiful Soup to automatically parse tables out of HTML field as DataFrame objects. 

In [60]:
tables = pd.read_html('../examples/fdic_failed_bank_list.html')

In [61]:
len(tables)

1

In [62]:
failures = tables[0]

In [63]:
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [64]:
close_timestamps = pd.to_datetime(failures['Closing Date'])

In [65]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

#### Parsing XML with lxml.objectify

XML (eXtensible Markup Language) is another common structures data format supporting hierarchical, nested data with metadata. 
XML and HTML are structured similar, but XML is more general. 

## Binary Data Formats

### Using HDF5 Format

### Reading Microsoft Excel Files

## Interacting with Web APIs

## Interacting with Databases